<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.2: Combinational Logic
**Prev: [Your First Chisel Module](2.1_first_module.ipynb)**<br>
**Next: [Control Flow](2.3_control_flow.ipynb)**

## Motivation
在本节中，你将看到如何使用 Chisel 组件实现组合逻辑。
我们将演示如何连接和操作三种基本的 Chisel 类型：`UInt` - 无符号整数；`SInt` - 有符号整数，以及 `Bool` - 真或假。
注意所有的 Chisel 变量都声明为 Scala 的 `val`。
永远不要为硬件结构使用 Scala 的 `var`，因为一旦定义，结构本身可能永远不会改变；只有在运行硬件时它的值才会改变。
Wires 可以用于参数化类型。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Common Operators
现在你已经了解了 `Module` 是如何构建的，让我们开始制作一些硬件吧！看看下面的空模块。

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

我们将我们的类命名为 `MyModule`，并且它扩展了 `Module`。这意味着它会映射到 Verilog 中的一个硬件模块。我们的 `MyModule` 模块有一个输入和一个输出。输入是一个4位的无符号整数（`UInt`），输出也是。

<span style="color:blue">**Example: Scala and Chisel Operators Look the Same**</span><br>
让我们看看可以对数据执行的不同操作。

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

我们创建了两个 `val`。第一个将两个 Scala `Int` 相加，因此 `println` 输出整数 2。第二个 `val` 将两个 *Chisel* `UInt` 相加，因此 `println` 将其视为硬件节点并打印出类型名称和指针（`chisel3.core.UInt@d`）。注意，`1.U` 是将 Scala `Int` (1) 转换为 Chisel `UInt` 字面量的类型转换。

我们需要将输出驱动到某个地方，所以我们现在只是将其连接到输入，就像在上一个教程中的 passthrough 模块一样。

<span style="color:blue">**Example: Incompatible Operation**</span><br>
如果我们将 Chisel 的 `1.U` 与字面值 `1` 相加会发生什么？这些类型是不兼容的，因为前者是值为 1 的硬件线，而后者是值为 1 的 Scala 值。因此，Chisel 会给出类型不匹配错误。

In [ ]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

进行操作时，记住类型之间的区别非常重要。Scala 是一种强类型语言，因此任何类型转换都必须显式进行。

<span style="color:blue">**Example: More Chisel Operators**</span><br>
其他常见操作是减法和乘法。这些操作在无符号整数上按预期处理。让我们来看一下这些操作的实际效果。我们展示 Verilog 代码，尽管有一些底层的 Chisel 特性使我们预期的简单代码变得有些模糊。

In [ ]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

以下是上述操作的示例测试器。与之前教程中使用匿名测试器类不同，我们将创建一个显式测试器类。这只是编写测试器的另一种方法。

In [ ]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

<span style="color:blue">**Example: Mux and Concatenation**</span><br>
除了加法、减法和乘法，Chisel 还有选择器（Mux）和连接操作符。如下所示。`Mux` 像传统的三元操作符一样操作，顺序为（选择，真值，假值）。注意，`true.B` 和 `false.B` 是创建 Chisel Bool 字面量的首选方式。`Cat` 的顺序是 MSB（最高有效位）然后是 LSB（最低有效位），并且只接受两个参数。连接多个值需要多个 `Cat` 调用或在后续部分中介绍的高级 Chisel 和 Scala 特性。

In [ ]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOperatorsTwo))

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}
println("SUCCESS!!")

注意，Verilog 中包含的是常量而不是实际的选择器或连接逻辑。这是因为 FIRRTL 转换简化了电路，消除了明显的逻辑。

要查看更完整的 Chisel 操作符列表，请参阅 [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)。有关操作符及其实现细节的最完整列表，请查看 [Chisel API](https://chisel-lang.org/api/latest/)。

---
# Exercises
要完成这些练习，你可能需要查看 [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)。

<span style="color:red">**Exercise: MAC**</span><br>
创建一个实现乘法累加功能 `(A*B)+C` 的 Chisel 模块，并通过测试平台。

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**Exercise: Arbiter**</span><br>
以下电路对来自 FIFO 的数据进行仲裁，发送到两个并行处理单元。FIFO 和处理单元（PEs）通过 ready-valid 接口进行通信。构建仲裁器，将数据发送到任何一个准备好接收数据的 PE，如果两个都准备好接收数据，则优先考虑 PE0。记住，当至少有一个 PE 可以接收数据时，仲裁器应该告诉 FIFO 它准备好接收数据。此外，等待一个 PE 确认它已准备好再确认数据是有效的。你可能需要使用二进制操作符来完成此练习。

<img src="images/arbiter.png" width="687" height="177">

In [ ]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

  ???
}

test(new Arbiter) { c =>
  import scala.util.Random
  val data = Random.nextInt(65536)
  c.io.fifo_data.poke(data.U)
  
  for (i <- 0 until 8) {
    c.io.fifo_valid.poke((((i >> 0) % 2) != 0).B)
    c.io.pe0_ready.poke((((i >> 1) % 2) != 0).B)
    c.io.pe1_ready.poke((((i >> 2) % 2) != 0).B)

    c.io.fifo_ready.expect((i > 1).B)
    c.io.pe0_valid.expect((i == 3 || i == 7).B)
    c.io.pe1_valid.expect((i == 5).B)
    
    if (i == 3 || i ==7) {
      c.io.pe0_data.expect((data).U)
    } else if (i == 5) {
      c.io.pe1_data.expect((data).U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Adder (Optional)**</span><br>
这个可选练习向你展示了 Chisel 最强大的功能之一，它的参数化能力。为了演示这一点，我们将构建一个参数化的加法器，当发生溢出时，它可以将输出饱和，或者截断结果（即循环）。

首先，看看下面的 `Module`。我们传递给它的参数称为 `saturate`，类型为*Scala* `Boolean`。这不是 Chisel 的 `Bool`。因此，我们不是创建一个可以饱和或截断的单一硬件加法器，而是创建一个*generator*，生成一个饱和硬件加法器或一个截断硬件加法器。这个决定是在编译时做出的。

接下来，注意输入和输出都是4位的 `UInt`。Chisel 有内置的宽度推断功能，如果你查看[速查表](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)，你会看到正常加法的位宽等于两个输入的最大位宽。这意味着

```scala
val sum = io.in_a + io.in_b
```

将使 `sum` 成为一个4位的导线，其值将是4位输入的截断结果。要检查加法是否应饱和，你需要将结果放在一个5位的导线中。这可以通过 `+&` 加法来完成，如速查表所示。

```scala
val sum = io.in_a +& io.in_b
```

最后，注意将一个4位的 `UInt` 导线连接到一个5位的 `UInt` 导线将默认截断最高有效位。你可以使用这一点轻松截断非饱和加法器的5位和。

In [ ]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  ???
}

for (saturate <- Seq(true, false)) {
  test(new ParameterizedAdder(saturate)) { c =>
    // 100 random tests
    val cycles = 100
    import scala.util.Random
    import scala.math.min
    for (i <- 0 until cycles) {
      val in_a = Random.nextInt(16)
      val in_b = Random.nextInt(16)
      c.io.in_a.poke(in_a.U)
      c.io.in_b.poke(in_b.U)
      if (saturate) {
        c.io.out.expect(min(in_a + in_b, 15).U)
      } else {
        c.io.out.expect(((in_a + in_b) % 16).U)
      }
    }
    
    // ensure we test saturation vs. truncation
    c.io.in_a.poke(15.U)
    c.io.in_b.poke(15.U)
    if (saturate) {
      c.io.out.expect(15.U)
    } else {
      c.io.out.expect(14.U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)